In [13]:
'''
Extracting grids with both phase and group disperion measurements
'''
# 提取vs ph gr 共有的点
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import interp1d
filepath_gr = './Output/dispersion_gr/'
filepath_ph = './Output/dispersion_ph/'
os.system('rm -rf  ./Output/disp_combine_gaussian_map_test && mkdir ./Output/disp_combine_gaussian_map_test')
os.system('rm -rf Figs && mkdir Figs' )
os.system('rm -rf ./Output/disp_pg_real && mkdir ./Output/disp_pg_real')
depth = np.array([2.5,5,10,15,20,30,40,60,80,100,120,150,180])
depth_mark = np.array([0,5,10,15,20,30,40,60,80,100,120,150,180])
depth = np.arange(0,150.5,0.5)
periods = np.array([8,10,12,14,16,18,20,22,24,26,28,30,32,35,40,45,50])
count = 0
for lat in np.arange(20,60,0.5):
    for lon in np.arange(70,140,0.5):
        key = "%.2f"%lat + "_"+"%.2f"%lon
        key_name = key + '.txt'
        file_gr = filepath_gr + key_name
        file_ph = filepath_ph + key_name
        if os.path.exists(file_ph) and os.path.exists(file_gr):
            temp = np.loadtxt(file_ph); freqs_p = temp[:,0]; vel_p = temp[:,1];error_p=temp[:,2]#[8-50]
            temp = np.loadtxt(file_gr); freqs_g = temp[:,0]; vel_g = temp[:,1];error_g=temp[:,2]#[8-50]
            if len(temp)>1:
                count =count +1
                # for using those data to predict dispersion curve comparing the vs from cnn
                tpg_out=np.array([periods,vel_p,error_p,vel_g,error_g]) # Test Data
                np.savetxt("./Output/disp_pg_real/"+"%.2f"%lat+"_"+"%.2f"%lon+".txt",tpg_out.T,fmt="%10.5f")
                # those data are first used to produce gaussian map and then input into the CNN to predict 1-D Vs
                # because we get measurement uncertainty involed here, we can do boostrap test during prediction 
                # process.
print(count)

4512 59.50_139.50.txt


In [9]:
def gaussian_map(vel,vel_axis,radius=0.1):
    rows  = vel_axis.shape[0]
    cols  = vel.shape[0]
    vel_map = np.zeros((rows,cols))
    for i in range(cols):
        vel_temp = vel[i]
        x_gaussian = gaussian(vel_temp, vel_axis, r=radius)
        vel_map[:, i] = x_gaussian
    return vel_map
#Note: The formula (1) in Hu et al. (2020) is wrong, the gaussian function is exp(-(a-b)^2/(2*r^2))
def gaussian(vel,vel_axis,r=0.1):
    x_gaussian = np.exp(-((vel_axis-vel)**2)/(2*r**2))
    return x_gaussian

In [11]:
# or used to directly gaussian map.
# if your data donot have uncertainty, you can directly generate gaussain map to perform CNN prediction.
vel_axis = np.linspace(1,5,num=60);r = 0.1
plot_flag=False;count = 0
fig=plt.figure(1,figsize=(10,4))
for lat in np.arange(20,60,0.5):
    for lon in np.arange(70,140,0.5):
        key = "%.2f"%lat + "_"+"%.2f"%lon
        key_name = key + '.txt'
        file_gr = filepath_gr + key_name
        file_ph = filepath_ph + key_name
        if os.path.exists(file_ph) and os.path.exists(file_gr):
            temp = np.loadtxt(file_ph); freqs_p = temp[:,0]; vel_p = temp[:,1];error_p=temp[:,2]#[8-50]
            temp = np.loadtxt(file_gr); freqs_g = temp[:,0]; vel_g = temp[:,1];error_g=temp[:,2]#[8-50]
            if len(temp)>1:
                count =count +1
#                 r = 0.1 or error_p or error_g
                vel_map_p= gaussian_map(vel_p,vel_axis,radius=r)
                vel_map_g= gaussian_map(vel_g,vel_axis,radius=r)
                name_pg = './Output/disp_combine_gaussian_map_test/c_' + "%.2f"%lat +"_"+"%.2f"%lon  + '.npy'
                np.save(name_pg,np.array([vel_map_p,vel_map_g]))
                if plot_flag:
                    plt.subplot(1,2,1)
                    plt.imshow(vel_map_p,aspect='auto',origin='lower',extent=[8,50,2,5])
                    plt.ylabel('Vs(km/s)')
                    plt.xlabel('T(s)')
                    plt.title('Phase velocity')
                    plt.subplot(1,2,2)
                    plt.imshow(vel_map_g,aspect='auto',origin='lower',extent=[8,50,2,5])
                    plt.ylabel('Vs(km/s)')
                    plt.xlabel('T(s)')
                    plt.title('Group velocity')
                    plt.tight_layout()
                    plt.pause(0.01)
                    plt.savefig('./Figs/'+key+'.jpg')
                    fig.clear()
print(count)

4512 59.50_139.50.txt


<Figure size 720x288 with 0 Axes>